In [ ]:
import numpy as np
T # trajectory , two colums matrix 
P # path follown, two colums matrix

lgT=len(T)
lgP=len(P)

depMin=min (T[0][0], P[0][0])
endMin=min (T[lgT][0], P[LgP][0])

# interval  of  computing is [depmin, endMin]

Subdivision =[]

